In [ ]:
import os

#insert filepath here
path = ""
os.chdir(path)

In [3]:
import pandas as pd
import numpy as np

In [224]:
res1_data = pd.read_csv("restaurant-1-orders.csv")
items = res1_data['Item Name'].unique()

In [7]:
def fillPD(data2015, items):

  zerosList = np.zeros(len(data2015.WeekOfYear.unique()))
  zerosList = zerosList.flatten().tolist()
  currentCols = data2015.columns.tolist()
  for i in items:
    if i not in currentCols:
      data2015[i] = zerosList

  return data2015
    

In [8]:
def add_prev_counts(dataNew, item):
    totals = []

    for index, row in dataNew.iterrows():

      week = int(row["WeekOfYear"])
      year = int(row["Year"]-1)
      df = dataNew[dataNew["Year"]== year]
      df = df[df["WeekOfYear"]== week]


      
      total = df[item].to_numpy()
      if total.shape[0] == 0:
        total = 0.0
      else:
        total = total[0]

      totals.append(total)
     
    return np.array(totals)

def add_prev_counts_test(dataOld,dataNew, item):
    totals = []
    dataOld = dataOld[dataOld["Year"]==2018]
    for index, row in dataNew.iterrows():

      week = int(row["WeekOfYear"])
 
   
      df = dataOld[dataOld["WeekOfYear"]== week]


      
      total = df[item].to_numpy()
      if total.shape[0] == 0:
        total = 0.0
      else:
        total = total[0]

      totals.append(total)
     
    return np.array(totals)

In [315]:
data2015 = pd.read_csv("WeeklyOrders2015.csv")
data2015 = data2015.fillna(0)
data2015 = fillPD(data2015, items)

In [349]:
data2016 = pd.read_csv("WeeklyOrders2016.csv")
data2016 = data2016.fillna(0)
data2016 = fillPD(data2016, items)

In [350]:
data2017 = pd.read_csv("WeeklyOrders2017.csv")
data2017 = data2017.fillna(0)
data2017 = fillPD(data2017, items)

In [351]:
data2018 = pd.read_csv("WeeklyOrders2018.csv")
data2018 = data2018.fillna(0)
data2018 = fillPD(data2018, items)

In [352]:
data2019 = pd.read_csv("WeeklyOrders2019.csv")
data2019 = data2019.fillna(0)
data2019 = fillPD(data2019, items)

In [353]:
trainData = pd.concat([data2015,data2016,data2017, data2018], axis = 0)
testData = data2019

In [355]:
Holidays = pd.read_csv("UK Holidays.csv")

def getHolidayCount(trainData, Holidays):

  listOfHolidays = Holidays.weekOfYear.unique()
  holidayCount = []
  for index, row in trainData.iterrows():

    week = int(row.WeekOfYear)
    p = Holidays[Holidays["weekOfYear"]==week]
    numHols = len(p["normalizeHolidayName"].unique())
    holidayCount.append(numHols)


  trainData["numHolidays"] = holidayCount

  return trainData
  

trainData = getHolidayCount(trainData, Holidays)
testData = getHolidayCount(testData, Holidays)

In [357]:
f = []
for i in trainData.columns.tolist():
  if i not in items:
    f.append(i)

In [42]:
def normalize(df,trainDF, items):
    result = df.copy()
    for feature_name in df.columns:
      if feature_name not in items:
        mean_value = trainDF[feature_name].mean()
        std_value = trainDF[feature_name].std()
        result[feature_name] = (df[feature_name] - mean_value) / std_value

      # if feature_name == "mark":
      #   result[feature_name] = (df[feature_name]/100)
    return result

In [361]:
trainDataN = normalize(trainData,trainData, items)
testDataN = normalize(testData,trainData, items)

In [362]:
f.pop(0)

'Year'

In [45]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
import tensorflow as tf

In [63]:
import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [381]:
from keras.callbacks import EarlyStopping
from tqdm import trange

es = EarlyStopping(monitor='val_loss', mode = "min",patience = 15)

MAEs = []
itemNames = []
for i in trange(len(items)):
  X_train = trainDataN[f].values
  y_train = trainDataN[items[i]].values
  featNum = trainDataN[items[i]].max()
  X_test = testDataN[f].values
  y_test = testDataN[items[i]].values

  t = add_prev_counts(trainDataN,items[i]).reshape(X_train.shape[0],1)
  X_train = np.append(X_train, t, axis = 1)


  t = add_prev_counts_test(trainDataN,testDataN, items[i]).reshape(X_test.shape[0],1)
  X_test = np.append(X_test, t, axis = 1)

  model = Sequential()
  model.add(Dense(14, input_dim=X_train.shape[1], kernel_initializer='he_uniform', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(featNum, kernel_initializer='he_uniform', activation='sigmoid'))
  model.add(Dense(1, kernel_initializer='he_uniform'))
  model.compile(loss = "mae", optimizer=tf.keras.optimizers.Adam(0.005))
  model.fit(X_train, y_train, verbose = 0, epochs = 350, validation_data = [X_test, y_test], callbacks = es)
  MAE = model.evaluate(X_test, y_test)
  print("MAE FOR:", items[i])
  print(MAE)
  print("\n")

  MAEs.append(MAE)
  itemNames.append(items[i])

  






  0%|          | 0/248 [00:00<?, ?it/s]

1/1 [==============================] - 0s 18ms/step - loss: 24.3481


  0%|          | 1/248 [00:03<14:49,  3.60s/it]

MAE FOR: Plain Papadum
24.34805679321289


1/1 [==============================] - 0s 18ms/step - loss: 0.1314


  1%|          | 2/248 [00:08<17:31,  4.28s/it]

...

MAE FOR: Methi - King Prawn
0.0003612142463680357


1/1 [==============================] - 0s 17ms/step - loss: 2.9419e-04


100%|█████████▉| 247/248 [09:52<00:02,  2.90s/it]

MAE FOR: Kurma
0.0002941898419521749


1/1 [==============================] - 0s 18ms/step - loss: 0.3896


100%|██████████| 248/248 [09:53<00:00,  2.39s/it]

MAE FOR: House Red wine 75cl
0.3895784914493561




In [382]:
zipped = list(zip(MAEs, itemNames))
MAE_DF = pd.DataFrame(zipped, columns=['MAE',"item"])

In [383]:
MAE_DF.MAE.mean()

1.497947523274272

ORDERS 2

In [32]:
trainData = pd.read_csv("orders2Train_clean.csv")
testData = pd.read_csv("orders2Test_clean.csv")

In [33]:
uniq = trainData.columns.tolist()
items_new = []

for i in items:

  if i in uniq:
    items_new.append(i)

In [34]:
items = items_new

In [35]:
len(items)

279

In [36]:
Holidays = pd.read_csv("UK Holidays.csv")

def getHolidayCount(trainData, Holidays):

  listOfHolidays = Holidays.weekOfYear.unique()
  holidayCount = []
  for index, row in trainData.iterrows():

    week = int(row.WeekOfYear)
    p = Holidays[Holidays["weekOfYear"]==week]
    numHols = len(p["normalizeHolidayName"].unique())
    holidayCount.append(numHols)


  trainData["numHolidays"] = holidayCount

  return trainData
  

trainData = getHolidayCount(trainData, Holidays)

testData = getHolidayCount(testData, Holidays)

In [39]:
f = []
for i in trainData.columns.tolist():
  if i not in items:
    f.append(i)

f.pop(0)
f.pop(0)

'Year'

In [43]:
trainDataN = normalize(trainData,trainData, items)
testDataN = normalize(testData,trainData, items)

In [47]:
from keras.callbacks import EarlyStopping
from tqdm import trange

es = EarlyStopping(monitor='val_loss', mode = "min",patience = 15)

MAEs = []
itemNames = []
for i in trange(len(items)):
  X_train = trainDataN[f].values
  y_train = trainDataN[items[i]].values
  featNum = trainDataN[items[i]].max()
  X_test = testDataN[f].values
  y_test = testDataN[items[i]].values

  t = add_prev_counts(trainDataN,items[i]).reshape(X_train.shape[0],1)
  X_train = np.append(X_train, t, axis = 1)


  t = add_prev_counts_test(trainDataN,testDataN, items[i]).reshape(X_test.shape[0],1)
  X_test = np.append(X_test, t, axis = 1)

  model = Sequential()
  model.add(Dense(14, input_dim=X_train.shape[1], kernel_initializer='he_uniform', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(featNum, kernel_initializer='he_uniform', activation='sigmoid'))
  model.add(Dense(1, kernel_initializer='he_uniform'))
  model.compile(loss = "mae", optimizer=tf.keras.optimizers.Adam(0.005))
  model.fit(X_train, y_train, verbose = 0, epochs = 350, validation_data = [X_test, y_test], callbacks = es)
  MAE = model.evaluate(X_test, y_test)
  print("MAE FOR:", items[i])
  print(MAE)
  print("\n")

  MAEs.append(MAE)
  itemNames.append(items[i])


  0%|          | 0/279 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step - loss: 3.2808


  0%|          | 1/279 [00:07<34:50,  7.52s/it]

MAE FOR: Tandoori Mixed Grill
3.2808449268341064


1/1 [==============================] - 0s 19ms/step - loss: 2.0678


  1%|          | 2/279 [00:09<19:50,  4.30s/it]

...

MAE FOR: Methi - king-prawn
0.0010216949740424752


1/1 [==============================] - 0s 20ms/step - loss: 0.4264


100%|█████████▉| 278/279 [10:55<00:02,  2.90s/it]

MAE FOR: House white wine 75cl
0.4264163374900818


1/1 [==============================] - 0s 20ms/step - loss: 0.0975


100%|██████████| 279/279 [11:01<00:00,  2.37s/it]

MAE FOR: House Red wine 75cl
0.09751506894826889




In [48]:
zipped = list(zip(MAEs, itemNames))
MAE_DF = pd.DataFrame(zipped, columns=['MAE',"item"])

In [49]:
MAE_DF.MAE.mean()

1.62348819972134

In [59]:
from sklearn.metrics import mean_absolute_error
baseline_maes = []
for index, row in MAE_DF.iterrows():
    i = row["item"]
    X_train = trainDataN[f].values
    y_train = trainDataN[i].values
    X_test = testDataN[f].values
    y_test = testDataN[i].values
    y_pred = np.ones(len(y_test))*np.mean(y_train)

    mae = mean_absolute_error(y_test, y_pred)
    baseline_maes.append(mae)



In [74]:
MAE_DF["BASELINE"] =  baseline_maes

MAE_DF.to_csv("mae_order2.csv")